In [5]:
exp_paths = [
    'exp_1', 'exp_5'
]
import pandas as pd
import xlwt
import os
seeds = [1, 2, 3, 4]
dataset_list = ['ohio', 'abc4d', 'ctr3_cgm_only', 'replace-bg',]
book = xlwt.Workbook(encoding='utf-8', style_compression=0)
dataset2sheet = {}
sheet = book.add_sheet('Main', cell_overwrite_ok=True)
metrics = ['rmse', 'mape', 'mae', 'grmse', 'time_lag']
num_mtc = len(metrics)
sheet.write(0 , 2 + len(dataset_list), 'ALL')
save_dict = {}
print_dataset = {
    'ohio': 'OHIO',
    'replace-bg': 'REPLACE-BG',
    'ctr3_cgm_only': 'CTR3',
    'abc4d': 'ABC4D',
}
print_metric = {
    'rmse' : 'RMSE',
    'mape' : 'MARD',
    'mae' : 'MAE',
    'grmse' : 'gRMSE',
    'time_lag': 'Time Lag',
}
exp2method = {
    'exp_1': 'LSTM',
    'exp_5' : 'GluADFL+LSTM',
}
version = 'coldstart_fl'
import numpy as np
for c, temp_experiment_version in enumerate(exp2method):
    seed_save_dict = []
    for seed in seeds:
        experiment_version = temp_experiment_version + '_seed_' + str(seed) if seed != 1 else temp_experiment_version
        
        save_dict = np.load(f'D:/code_log/{version}/{experiment_version}/{experiment_version}.npy', allow_pickle=True)[()]
        seed_save_dict.append(save_dict)
    
    mean_save_dict = {}
    for save_dict in seed_save_dict:
        for key in save_dict:
            if key not in mean_save_dict:
                mean_save_dict[key] = [[save_dict[key][0]], [save_dict[key][1]]]
            else:
                mean_save_dict[key][0].append(save_dict[key][0])
                mean_save_dict[key][1].append(save_dict[key][1])
    for key in mean_save_dict:
        temp_mean = np.mean(mean_save_dict[key][0])
        temp_sd1 = np.std(mean_save_dict[key][0])
        temp_sd2 = np.mean(mean_save_dict[key][1])
        mean_save_dict[key] = (temp_mean, temp_sd1, temp_sd2)
    
    for r, load_dataset in enumerate(dataset_list):

        sheet.write(1 + num_mtc * r , 0, print_dataset[load_dataset])

        for c, test_dataset in enumerate(dataset_list):
            sheet.write(0, 2 + c, print_dataset[test_dataset])
       
            for n, metric in enumerate(metrics):
                print_str = f'{mean_save_dict[(load_dataset, test_dataset, metric)][0]:.2f}({mean_save_dict[(load_dataset, test_dataset, metric)][2]:.2f})'
                sheet.write(1 + num_mtc * r + n, 2 + c, print_str)
                sheet.write(1 + num_mtc * r + n, 1, print_metric[metric])

        for n, metric in enumerate(metrics):
            print_str = f'{mean_save_dict[(load_dataset, metric)][0]:.2f}({mean_save_dict[(load_dataset, metric)][2]:.2f})'
            sheet.write(1 + num_mtc * r + n, 2 + len(dataset_list), print_str)
            


        book.save(f'{temp_experiment_version}/{temp_experiment_version}_merge_seeds.xls')